In [31]:
import json
import csv
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
import string
import ast
import gensim 
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from operator import itemgetter

In [32]:
raw_file_str = "../food-com-recipes-and-user-interactions/RAW_recipes.csv"
df_file = pd.read_csv(raw_file_str)

In [33]:
df_file.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [72]:
# Using some stopwords from https://github.com/AlludedCrabb/sound-tasty
cooking_stop_words = list(set([
        'canned', 'cans', 'drained', 'and', 'halved', 'cup', 'cups',
        'teaspoon', 'tablespoon', 'teaspoons', 'tablespoons',
        'finely', 'freshly', 'fresh', 'thickcut', 'to', 'taste',
        'grated', 'cut', 'into', 'wedges', 'pounds', 'unpeeled', 'large',
        'minced', 'slice', 'slices', 'sliced', 'thick-cut', 'cut',
        'crosswise', 'pieces', 'toothpicks', 'low-fat', 'chopped', 'or',
        'taste', 'cooked', 'dry', 'shredded', 'beaten', 'dried', 'melted',
        'stems', 'removed', 'diced', 'ounce', 'ounces', 'packages',
        'softened', 'such', 'RedHot®', 'RedHot', 'Franks', "Frank's",
        'crumbled', 'Old', 'Bay®', 'Bay', 'pinch', 'for', 'garnish', 'slice',
        'slices', 'needed', 'inch', 'cubes', 'cooking', 'spray', 'ground',
        'rotisserie', 'lowfat', 'as', 'quarteres', 'cloves', 'more', 'can',
        'package', 'frozen', 'thawed', 'packet', 'reducedfat', 'Knorr',
        'container', 'pound', 'peeled', 'deveined', 'seeded', 'ripe',
        'English', 'juiced', 'plus', 'more', 'Hass', 'cubed', 'Mexicanstyle',
        'hearts', 'prepared', 'party', 'pitted', 'mashed',
        'roma', 'optional', 'chunk', 'Hot', 'bunch', 'cleaned', 'box',
        'chickenflavored', 'Golden', 'delicious', 'cored', 'any', 'flavor',
        'flavored', 'whole', 'allpurpose', 'all', 'purpose', 'deep', 'frying',
        'dash', 'packed', 'in', 'French', 'jar', 'small', 'head', 'little',
        'smokie', 'seasoned', 'Boston', 'Bibb', 'leaves', 'lean', 'pickled',
        'Asian', 'dark', 'flaked', 'rolled', 'packed', 'jellied',
        'thirds', 'with', 'attached', 'skewers', 'skinless', 'boneless',
        'half', 'kernels', 'rinsed', 'quart', 'quarts', 'kernel',
        'Italianstyle', 'unpopped', 'lightly', 'coating', 'SAUCE',
        'lengthwise', 'miniature', 'semisweet', 'rinsed', 'round',
        'squeezed', 'stewed', 'raw', 'the', 'liquid', 'reserved', 'medium',
        'instant', 'solid', 'pack', 'refrigerated', 'halves', 'distilled',
        'loaf', 'extra', 'virgin', 'crushed', 'kosher', 'toasted', 'buttery',
        'TM', 'panko', 'Japanese', 'regular', 'bottle', 'bottles', 'thin',
        'peel', 'paper', 'thick', 'circles', 'unbleached',
        'breast', 'breasts', 'wings', 'strips', 'jumbo', 'giant', 'chunks',
        'quickcooking', 'sweetened', 'flakes', 'Ranchstyle', 'snipped',
        'food', 'ROTEL', 'Italian', 'sticks', 'stick', 'crescent', 'thinly',
        'boiled', 'Genoa', 'roasted', 'thin', 'extrasharp', 'pressed',
        'sifted', 'split', 'tips', 'discarded', 'mini', 'deli', 'drain',
        'reserve', 'diameter', 'Greek', 'Thai', 'drops', 'square', 'crusty',
        'American', 'selfrising', 'imitation', 'Wings', 'apart', 'at',
        'joints', 'wing', 'tips', 'discarded', 'parts',
        'tops', 'seperated', 'blend', 'coarsely', 'sweet', 'stalk', 'heads',
        'husked', 'divided', 'pats', 'unsalted', 'active', 'warm', 'sea',
        'separated', 'herb', 'overripe', 'degrees', 'F', 'C', 'room',
        'temperature', 'machine', 'very', 'pint', 'puree', 'coarse',
        'envelopes', 'lukewarm', 'creamstyle', 'unsweetened',
        'lite', 'of', 'chilled', 'freezer', 'cold', 'brushing', 'nonfat',
        'squares', 'tails', 'thigh', 'quarters', 'Masterpiece', 'KC', 'from',
        'El', 'Paso', 'bulk', 'Hunts', 'Roma', 'light', 'fluid', 'lagerstyle',
        'stalks', 'quartered', 'undrained', 'drained', 'Tony', 'Chacheres',
        'lump', 'uncooked', 'cube', 'bits', 'hair', 'angel', 'trimmed',
        'stew', 'spaghetti', 'brisket', 'bitesized', 'matchstick', 'Chobani',
        'unbaked', 'crust', 'torn', 'bonein', 'pounded', 'bitesize',
        'granules', 'boiling', 'yolk', 'coloring', 'pinch', 'a', 'blender',
        'fine', 'which', 'extralarge', 'use', 'will', 'make', 'garnish',
        'barely', 'moistened', 'about', 'right', 'before', 'serving', 'mix',
        
    ]))
cooking_stop_words

['coloring',
 'RedHot®',
 'half',
 'and',
 'small',
 'pickled',
 'pats',
 'regular',
 'jellied',
 'lukewarm',
 'mashed',
 'stalk',
 'apart',
 'lagerstyle',
 'breasts',
 'juiced',
 'degrees',
 'TM',
 'light',
 'dash',
 'slice',
 'in',
 'halves',
 'extrasharp',
 'frozen',
 'raw',
 'or',
 'cube',
 'peel',
 'parts',
 'warm',
 'crescent',
 'divided',
 'skewers',
 'buttery',
 'KC',
 "Frank's",
 'Chacheres',
 'delicious',
 'little',
 'imitation',
 'Bibb',
 'large',
 'Genoa',
 'garnish',
 'squares',
 'leaves',
 'thinly',
 'crumbled',
 'blend',
 'active',
 'Masterpiece',
 'Golden',
 'thick',
 'pint',
 'Knorr',
 'freshly',
 'joints',
 'tips',
 'finely',
 'distilled',
 'unsweetened',
 'Bay®',
 'ounce',
 'rinsed',
 'chopped',
 'Bay',
 'quart',
 'toasted',
 'Wings',
 'bitesize',
 'SAUCE',
 'chunks',
 'ripe',
 'refrigerated',
 'Old',
 'kernels',
 'deli',
 'jar',
 'lengthwise',
 'teaspoons',
 'unpeeled',
 'selfrising',
 'cleaned',
 'paper',
 'ground',
 'food',
 'seperated',
 'thickcut',
 'deveined',


In [87]:
unhelpful = ['fresh', 'ripe', 'cracked', 'cooking', 'coarse', 'light', 
             'mild', 'hot', 'minced', 'dark roast', 'unsifted', 'canned', 
             'cans', 'drained', 'halved', 'finely', 'freshly', 'thickcut', 
             'grated', 'cut', 'unpeeled', 'large', 'minced', 'slice', 
             'slices', 'sliced', 'chopped','shredded', 'beaten', 'dried', 
             'melted', 'stems', 'softened', 'packages', 'crumbled', 'ground',
             'low-fat', 'rotisserie', 'lowfat', 'can', 'thawed', 'packet', 
             'reducedfat', 'small', 'pats', 'regular', 'lukewarm', 'mashed', 
             'stalk', 'breast', 'breasts', 'juiced', 'halves', 'extrasharp', 
             'sharp', 'extra sharp', 'frozen', 'raw', 'warm', 'divided', 
             'little', 'squares', 'thinly', 'thick', 'rinsed', 'toasted', 
             'bitesize', 'chunks', 'refrigerated', 'kernel', 'kernels', 
             'jar', 'lengthwise', 'unpeeled', 'cleaned', 'paper', 'melted', 
             'separated', 'seperated', 'deveined', 'party', 'bunch', 'overripe', 
             'boiled', 'chunk', 'container', 'bitesized', 'sweet', 'strips', 
             'sifted', 'roma', 'very', 'undrained', 'stewed', 'thawed', 'lean', 
             'roasted', 'extra', 'lite', 'coarsely', 'pressed', 'square', 'jumbo', 
             'yolk', 'yolks', 'barely', 'pitted', 'cored', 'puree', 'cubes', 'angel', 
             'hair', 'angelhair', 'giant', 'husked', 'chilled', 'thigh', 'trimmed', 
             'thin', 'lightly', 'cubed', 'drops', 'grated', 'boneless', 'unsalted', 
             'pieces', 'skinless', 'pounded', 'chickenflavored', 'extralarge', 
             'medium', 'reserve', 'unbaked', 'crushed', 'wings', 'crosswise', 
             'cold', 'bonein', 'bone in', 'squeezed', 'kosher', 'miniature', 'tails', 
             'quarters', 'attached', 'loaf', 'dry', 'more', 'head', 'removed', 'packed', 
             'hearts', 'matchstick', 'unbleached', 'heads', 'stems', 'sea', 'diced', 
             'mini', 'cut', 'unpopped', 'box', 'uncooked', 'freezer', 'stalks', 
             'shredded', 'halved', 'snipped', 'thick-cut', 'split', 'seeded', 
             'sweetened', 'discarded', 'lump', 'boiling', 'whole', 'semisweet', 
             'semi-sweet', 'quartered', 'moistened', 'reserved', 'prepared']

In [88]:
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list += unhelpful
lemmatizer = WordNetLemmatizer()

In [35]:
def _flatten(list_of_lists):
    for x in list_of_lists:
        if hasattr(x, '__iter__') and not isinstance(x, str):
            for y in _flatten(x):
                yield y
        else:
            yield x

In [36]:
test= df_file['ingredients'][1]
test

"['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']"

In [37]:
x = ast.literal_eval(test)
x

['prepared pizza crust',
 'sausage patty',
 'eggs',
 'milk',
 'salt and pepper',
 'cheese']

In [38]:
x[0]

'prepared pizza crust'

In [39]:
x2 = [n.strip() for n in x]

In [40]:
x2

['prepared pizza crust',
 'sausage patty',
 'eggs',
 'milk',
 'salt and pepper',
 'cheese']

In [41]:
flattened = _flatten(df_file['ingredients'])

In [42]:
list(flattened)

["['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",
 "['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']",
 "['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']",
 "['spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'olive oil', 'red wine vinegar', 'salt', 'pepper', 'red bell pepper', 'yellow bell pepper']",
 "['tomato juice', 'apple cider vinegar', 'sugar', 'salt', 'pepper', 'clove oil', 'cinnamon oil', 'dry mustard']",
 "['milk', 'vanilla ice cream', 'frozen apple juice concentrate', 'apple']",
 "['fennel seeds', 'green olives', 'ripe olives', 'garlic', 'peppercorn', 'orange rind', 'orange juice', 'red chile', 'extra virgin olive oil']",
 "['pork spareribs', 'soy sauce', 'fresh garlic', 'fresh ginger', 'chili powder', 'f

In [43]:
big_list = []
for row in df_file['ingredients']:
    big_list.append(ast.literal_eval(row))
    

In [44]:
tags = []
for row in df_file['tags']:
    tags.append(ast.literal_eval(row))

list_tags = list(_flatten(tags))
list_tags

['60-minutes-or-less',
 'time-to-make',
 'course',
 'main-ingredient',
 'cuisine',
 'preparation',
 'occasion',
 'north-american',
 'side-dishes',
 'vegetables',
 'mexican',
 'easy',
 'fall',
 'holiday-event',
 'vegetarian',
 'winter',
 'dietary',
 'christmas',
 'seasonal',
 'squash',
 '30-minutes-or-less',
 'time-to-make',
 'course',
 'main-ingredient',
 'cuisine',
 'preparation',
 'occasion',
 'north-american',
 'breakfast',
 'main-dish',
 'pork',
 'american',
 'oven',
 'easy',
 'kid-friendly',
 'pizza',
 'dietary',
 'northeastern-united-states',
 'meat',
 'equipment',
 'time-to-make',
 'course',
 'preparation',
 'main-dish',
 'chili',
 'crock-pot-slow-cooker',
 'dietary',
 'equipment',
 '4-hours-or-less',
 '60-minutes-or-less',
 'time-to-make',
 'course',
 'main-ingredient',
 'preparation',
 'occasion',
 'side-dishes',
 'eggs-dairy',
 'potatoes',
 'vegetables',
 'oven',
 'easy',
 'dinner-party',
 'holiday-event',
 'easter',
 'cheese',
 'stove-top',
 'dietary',
 'christmas',
 'new-ye

In [86]:
big_list

[['winter squash',
  'mexican seasoning',
  'mixed spice',
  'honey',
  'butter',
  'olive oil',
  'salt'],
 ['prepared pizza crust',
  'sausage patty',
  'eggs',
  'milk',
  'salt and pepper',
  'cheese'],
 ['ground beef',
  'yellow onions',
  'diced tomatoes',
  'tomato paste',
  'tomato soup',
  'rotel tomatoes',
  'kidney beans',
  'water',
  'chili powder',
  'ground cumin',
  'salt',
  'lettuce',
  'cheddar cheese'],
 ['spreadable cheese with garlic and herbs',
  'new potatoes',
  'shallots',
  'parsley',
  'tarragon',
  'olive oil',
  'red wine vinegar',
  'salt',
  'pepper',
  'red bell pepper',
  'yellow bell pepper'],
 ['tomato juice',
  'apple cider vinegar',
  'sugar',
  'salt',
  'pepper',
  'clove oil',
  'cinnamon oil',
  'dry mustard'],
 ['milk', 'vanilla ice cream', 'frozen apple juice concentrate', 'apple'],
 ['fennel seeds',
  'green olives',
  'ripe olives',
  'garlic',
  'peppercorn',
  'orange rind',
  'orange juice',
  'red chile',
  'extra virgin olive oil'],
 [

In [90]:
big_list[0]

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

In [105]:
tknz = []
for item in big_list[0]:
    tknz.append(word_tokenize(item))
list(_flatten(tknz))

['winter',
 'squash',
 'mexican',
 'seasoning',
 'mixed',
 'spice',
 'honey',
 'butter',
 'olive',
 'oil',
 'salt']

In [98]:
test2 = _flatten(big_list[0])
list(big_list[0])

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

In [99]:
big_list[0]

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

In [110]:
token_recipes = []
for recipe in big_list:
    tokens = [word_tokenize(word) for word in recipe]
    flat_tkns = list(_flatten(tokens))
    stopped_tokens = [w.lower() for w in flat_tkns if w not in stopwords_list]
    lemma_tokens = [lemmatizer.lemmatize(token) for token in stopped_tokens]
    token_recipes.append(lemma_tokens)

In [111]:
token_recipes

[['winter',
  'squash',
  'mexican',
  'seasoning',
  'mixed',
  'spice',
  'honey',
  'butter',
  'olive',
  'oil',
  'salt'],
 ['pizza',
  'crust',
  'sausage',
  'patty',
  'egg',
  'milk',
  'salt',
  'pepper',
  'cheese'],
 ['beef',
  'yellow',
  'onion',
  'tomato',
  'tomato',
  'paste',
  'tomato',
  'soup',
  'rotel',
  'tomato',
  'kidney',
  'bean',
  'water',
  'chili',
  'powder',
  'cumin',
  'salt',
  'lettuce',
  'cheddar',
  'cheese'],
 ['spreadable',
  'cheese',
  'garlic',
  'herb',
  'new',
  'potato',
  'shallot',
  'parsley',
  'tarragon',
  'olive',
  'oil',
  'red',
  'wine',
  'vinegar',
  'salt',
  'pepper',
  'red',
  'bell',
  'pepper',
  'yellow',
  'bell',
  'pepper'],
 ['tomato',
  'juice',
  'apple',
  'cider',
  'vinegar',
  'sugar',
  'salt',
  'pepper',
  'clove',
  'oil',
  'cinnamon',
  'oil',
  'mustard'],
 ['milk', 'vanilla', 'ice', 'cream', 'apple', 'juice', 'concentrate', 'apple'],
 ['fennel',
  'seed',
  'green',
  'olive',
  'olive',
  'garlic

In [ ]:
big_list_tokenized = [[lemmatizer.lemmatize(w) for w in token] for token in stopped_tokens]

In [45]:
flattened = _flatten(big_list)

In [46]:
one_list = list(flattened)

In [47]:
one_list

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt',
 'prepared pizza crust',
 'sausage patty',
 'eggs',
 'milk',
 'salt and pepper',
 'cheese',
 'ground beef',
 'yellow onions',
 'diced tomatoes',
 'tomato paste',
 'tomato soup',
 'rotel tomatoes',
 'kidney beans',
 'water',
 'chili powder',
 'ground cumin',
 'salt',
 'lettuce',
 'cheddar cheese',
 'spreadable cheese with garlic and herbs',
 'new potatoes',
 'shallots',
 'parsley',
 'tarragon',
 'olive oil',
 'red wine vinegar',
 'salt',
 'pepper',
 'red bell pepper',
 'yellow bell pepper',
 'tomato juice',
 'apple cider vinegar',
 'sugar',
 'salt',
 'pepper',
 'clove oil',
 'cinnamon oil',
 'dry mustard',
 'milk',
 'vanilla ice cream',
 'frozen apple juice concentrate',
 'apple',
 'fennel seeds',
 'green olives',
 'ripe olives',
 'garlic',
 'peppercorn',
 'orange rind',
 'orange juice',
 'red chile',
 'extra virgin olive oil',
 'pork spareribs',
 'soy sauce',
 'fresh garlic',
 'fresh gin

In [48]:
len(one_list)

2096582

In [49]:
uniques = set(one_list)

In [50]:
len(list(uniques))

14942

In [51]:
uniques

{'green tomatoes',
 'rice krispies treats',
 'veal steak',
 'boneless skinless chicken cutlets',
 'caramel extract',
 'canned chocolate pie filling',
 'lettuce head',
 'celery rib',
 'green apple',
 'toasted rice cereal',
 'sharp processed cheese spread',
 'roasted unsalted peanuts',
 'cayenne',
 'malt-o-meal original hot cereal',
 'skirt steaks',
 'neapolitan ice cream',
 'pita chips',
 'jalapeno potato chips',
 'ground moose',
 'goji berries',
 'boneless steak',
 'instant coffee crystals',
 'canned white beans',
 'lollipop sticks',
 'green pepper strip',
 'triscuit parmesan garlic thin crisps',
 'powdered drink mix',
 'firm ripe peach',
 'pink grapefruit, juice of',
 'ragu parmesan and romano spaghetti sauce',
 'gingersnaps',
 'honey roasted peanuts',
 'low-sodium bouillon cube',
 'fresh rhubarb',
 'beef kidney',
 'halloumi cheese',
 'blackberries packed in water',
 'dried broad beans',
 'beef flank steak',
 'black cherry rum',
 'instant roux mix',
 'margarita jell-o gelatin',
 'appl

In [52]:
with open('../write_data/unique_ingred.txt', mode='w', encoding='utf-8') as myfile:
    for ingred in list(uniques):
        myfile.write('%s\n' % ingred)

In [53]:
def process_article(article):
    tokens = nltk.word_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in stopwords_list]
    return stopwords_removed 

In [54]:
ingred_freqdist = FreqDist(one_list)

In [55]:
ingred_freqdist.most_common(200)

[('salt', 85746),
 ('butter', 54975),
 ('sugar', 44535),
 ('onion', 39065),
 ('water', 34914),
 ('eggs', 33761),
 ('olive oil', 32822),
 ('flour', 26266),
 ('milk', 25786),
 ('garlic cloves', 25748),
 ('pepper', 22319),
 ('brown sugar', 18655),
 ('garlic', 18087),
 ('all-purpose flour', 17659),
 ('baking powder', 17504),
 ('egg', 17304),
 ('salt and pepper', 15415),
 ('parmesan cheese', 14807),
 ('lemon juice', 14233),
 ('baking soda', 14099),
 ('vegetable oil', 13912),
 ('vanilla', 13315),
 ('black pepper', 13098),
 ('cinnamon', 12560),
 ('tomatoes', 11950),
 ('sour cream', 11779),
 ('garlic powder', 10887),
 ('vanilla extract', 10271),
 ('oil', 9925),
 ('honey', 9898),
 ('onions', 9872),
 ('cream cheese', 9827),
 ('garlic clove', 9779),
 ('celery', 9541),
 ('cheddar cheese', 8969),
 ('unsalted butter', 8935),
 ('soy sauce', 8856),
 ('mayonnaise', 8736),
 ('paprika', 7982),
 ('chicken broth', 7963),
 ('worcestershire sauce', 7832),
 ('extra virgin olive oil', 7704),
 ('fresh parsley',

In [56]:
ingred_freqdist.most_common()[:500:-1]

[('nepitella', 1),
 ('skinless halibut fillet', 1),
 ("hunt's fire roasted diced tomatoes", 1),
 ('rosarita premium whole black beans', 1),
 ('apricot jell-o', 1),
 ('seasoned rice vinegar with roasted garlic', 1),
 ('roasted red pepper pesto', 1),
 ('pesto garlic tortillas', 1),
 ('thousand island sandwich spread', 1),
 ('low-fat low-sodium condensed cream of tomato soup', 1),
 ('red pepper rings', 1),
 ('raspberry-flavored white wine vinegar', 1),
 ('stirrings bloody mary cocktail mixer', 1),
 ('southwestern-flavored cornbread stuffing mix', 1),
 ('kala namak', 1),
 ('lean pot roast beef', 1),
 ('onion bread', 1),
 ('nonfat berry yogurt', 1),
 ('fat-free apricot yogurt', 1),
 ('low-fat condensed cheddar cheese soup', 1),
 ('cinnamon dolce syrup', 1),
 ('smoked jalapeno sausage', 1),
 ('meatloaf recipe', 1),
 ('wheat grass powder', 1),
 ('sugar-free cranberry flavor gelatin', 1),
 ('gourd', 1),
 ('yigandes beans', 1),
 ('polished rice', 1),
 ('vanilla cupcake', 1),
 ('birds eye frozen

In [57]:
def count_vectorize(ingredients, vocab=None):
    if vocab:
        unique_words = vocab
    else:
        unique_words = list(set(ingredients))
    
    ingred_dict = {i:0 for i in unique_words}
    
    for word in ingredients:
        ingred_dict[word] += 1
        
    return ingred_dict

In [58]:
test = count_vectorize(one_list)
test

{'green tomatoes': 119,
 'rice krispies treats': 2,
 'veal steak': 2,
 'boneless skinless chicken cutlets': 11,
 'caramel extract': 2,
 'canned chocolate pie filling': 1,
 'lettuce head': 7,
 'celery rib': 682,
 'green apple': 100,
 'toasted rice cereal': 13,
 'sharp processed cheese spread': 1,
 'roasted unsalted peanuts': 34,
 'cayenne': 1855,
 'malt-o-meal original hot cereal': 1,
 'skirt steaks': 95,
 'neapolitan ice cream': 3,
 'pita chips': 17,
 'jalapeno potato chips': 1,
 'ground moose': 3,
 'goji berries': 6,
 'boneless steak': 4,
 'instant coffee crystals': 57,
 'canned white beans': 13,
 'lollipop sticks': 14,
 'green pepper strip': 10,
 'triscuit parmesan garlic thin crisps': 1,
 'powdered drink mix': 20,
 'firm ripe peach': 2,
 'pink grapefruit, juice of': 2,
 'ragu parmesan and romano spaghetti sauce': 5,
 'gingersnaps': 62,
 'honey roasted peanuts': 30,
 'low-sodium bouillon cube': 1,
 'fresh rhubarb': 92,
 'beef kidney': 2,
 'halloumi cheese': 69,
 'blackberries packed 

In [59]:
for key, value in sorted(test.items(), key = itemgetter(1), reverse = True):
    print(key, value)

salt 85746
butter 54975
sugar 44535
onion 39065
water 34914
eggs 33761
olive oil 32822
flour 26266
milk 25786
garlic cloves 25748
pepper 22319
brown sugar 18655
garlic 18087
all-purpose flour 17659
baking powder 17504
egg 17304
salt and pepper 15415
parmesan cheese 14807
lemon juice 14233
baking soda 14099
vegetable oil 13912
vanilla 13315
black pepper 13098
cinnamon 12560
tomatoes 11950
sour cream 11779
garlic powder 10887
vanilla extract 10271
oil 9925
honey 9898
onions 9872
cream cheese 9827
garlic clove 9779
celery 9541
cheddar cheese 8969
unsalted butter 8935
soy sauce 8856
mayonnaise 8736
paprika 7982
chicken broth 7963
worcestershire sauce 7832
extra virgin olive oil 7704
fresh parsley 7656
cornstarch 7486
fresh ground black pepper 7160
carrots 7023
parsley 7001
chili powder 6984
bacon 6948
ground cinnamon 6864
carrot 6707
potatoes 6507
nutmeg 6299
cayenne pepper 6285
granulated sugar 6254
ground cumin 6169
ground beef 5824
green onions 5814
red onion 5777
walnuts 5765
pecans 57

honeydew melon 131
italian spices 131
msg 131
colby cheese 131
flat-leaf italian parsley 131
sausages 131
fresh blackberries 130
strawberry gelatin 130
vital wheat gluten 130
almond meal 130
safflower oil 130
baking potato 130
cooked long-grain rice 130
whole allspice 130
pickling salt 130
oysters 129
flax seeds 129
country-style pork ribs 129
fruit cocktail 129
ground oregano 129
shiitake mushroom 129
baby portabella mushrooms 129
coconut flakes 129
thin spaghetti 129
sage leaves 129
arrowroot 129
sweet basil 129
manicotti 129
hot chili sauce 129
sandwich buns 128
boneless pork shoulder 128
spices 128
fat-free cheddar cheese 128
diced pimentos 128
chambord raspberry liquor 128
sugar snap peas 128
almond flour 128
quick-cooking tapioca 128
catfish fillets 128
blanched almonds 128
lime wedges 128
french dressing 128
white balsamic vinegar 127
mushroom soup 127
pork spareribs 127
cayenne powder 127
caramel sauce 127
frosting 127
madeira wine 126
parsley sprig 126
italian tomatoes 126
tam

dried italian herb seasoning 80
smoked turkey 80
mint sprigs 80
fresh thyme sprig 80
seasoned pepper 80
double-acting baking powder 80
courgettes 80
green leaf lettuce 80
dried chilies 80
condensed beef broth 80
relish 80
grits 79
cream-style cottage cheese 79
green onion top 79
emeril's original essence 79
cardamom seed 79
mild curry powder 79
focaccia bread 79
limoncello 79
dry pectin 79
vinaigrette 79
blueberry pie filling 79
gluten 79
albacore tuna 78
refrigerated pie crust 78
spiced rum 78
tortellini 78
sweet unsalted butter 78
fresh lemongrass 78
white shoepeg corn 78
green cardamoms 78
merlot 78
skinless chicken pieces 78
halibut steaks 78
butter lettuce 78
mini chocolate chips 78
vegetable bouillon cubes 78
sole fillets 78
fudge brownie mix 78
7-up soda 78
madras curry powder 78
coleslaw 78
butter recipe cake mix 78
chicken flavor stuffing mix 78
raspberry jell-o gelatin 77
semisweet chocolate morsel 77
chili paste with garlic 77
hot dog 77
pretzel 77
boneless pork 77
chicken c

browning sauce 49
tartar sauce 49
agave syrup 49
sorghum flour 49
sofrito sauce 49
bouillon 49
instant beef bouillon 49
cod fish fillet 49
black bean sauce 49
frozen pearl onions 49
35% cream 49
hot chili peppers 49
frozen peaches 49
dried rubbed sage 49
gram flour 49
raisin bran cereal 49
chocolate instant pudding 49
diced green chilis 49
cherry gelatin 49
fresh cilantro stems 49
colored sprinkles 49
lime gelatin 49
cooked elbow macaroni 49
smoked cheddar cheese 49
reduced-fat milk 49
frozen broccoli carrots cauliflower mix 49
sage leaf 49
small white beans 49
stout beer 49
ancho chilies 49
boneless pork butt 48
frozen okra 48
cracker crumbs 48
frozen french fries 48
brown basmati rice 48
pork baby back ribs 48
cheese whiz 48
rabbit 48
french haricots vert 48
green onions with tops 48
tuna in vegetable oil 48
lebanese cucumber 48
grape leaves 48
green peppercorn 48
filet mignon 48
seitan 48
pepper sauce 48
dry roasted salted peanut 48
self-rising cornmeal 48
sumac 48
king prawns 48
sw

2% evaporated milk 26
dipping chocolate 26
best foods mayonnaise 26
hidden valley original ranch dips mix 26
serrano ham 26
cheese slice 26
anchovy 26
pork mince 26
sesame 26
dried mango 26
fenugreek leaves 26
bing cherries 26
fresh button mushrooms 26
tart green apples 26
mutton 26
lean pork chops 26
godiva original chocolate liqueur 26
country-style dijon mustard 26
ginger beer 26
frying oil 26
blood oranges 26
japanese eggplant 26
top round steak 26
star fruit 26
passion fruit pulp 26
boneless ham 26
refrigerated cheese tortellini 26
clam 26
frozen white bread dough 26
lychees 26
cheez-it crackers 26
hickory smoked bacon 26
gravy mix 26
canned solid-pack pumpkin 26
frozen peppers and onions 26
miniature pretzel twists 26
boiled ham 26
cabbage and carrot coleslaw mix 26
dry green lentils 26
potato water 26
tomatoes seasoned with basil garlic & oregano 26
small mushrooms 26
garlic chives 26
beet juice 26
white breadcrumb 26
hot dog bun 26
adzuki beans 25
dry white vermouth 25
light oi

...

english toffee syrup 3
frozen broccoli and cauliflower 3
caperberries 3
garlic balsamic & olive oil diced tomatoes 3
chipotle cheddar cheese 3
pineapple tidbits in syrup 3
iceberg lettuce leaf 3
hog casings 3
cherry frosting 3
rock cornish hens 3
stravecchio cheese 3
betty crocker super moist lemon cake mix 3
beef medallions 3
kraft shredded cheddar cheese 3
southeast asian fish sauce 3
cinnamon raisin bagel 3
pistachio ice cream 3
grape gelatin 3
roasted potatoes 3
mahi mahi steaks 3
herb salad dressing mix 3
jelly sponge rolls 3
preserved radish 3
spam lite 3
chocolate swiss rolls 3
sugar-free instant white chocolate pudding mix 3
tempura flour 3
tarama 3
nilla wafer crumbs 3
prepared onion dip 3
perch 3
vanilla tea bags 3
red serrano chilie 3
reduced-fat sausage 3
baileys caramel irish cream 3
orange flavoring 3
caramel icing 3
olive oil spread 3
canned spaghetti and sauce 3
dried craisin 3
crisco baking sticks butter flavor all-vegetable shortening 3
wood chunks 3
injera bread 3
am

cremora 2
orange paste food coloring 2
kosher pickle 2
seafood salad 2
st. germain elderflower liqueur 2
oat bran flakes 2
canned mango 2
low-fat ice cream 2
sugar-free jam 2
sea legs 2
dried guajillo chiles 2
monosodium glutamate 2
fat-free balsamic vinegar 2
white cheese dip 2
red chili seeds 2
waxed yellow beans 2
club house "lagrille" vegetable seasoning 2
potato pierogies 2
strawberry muffin mix 2
sugar-free apricot preserves 2
pillsbury plus yellow cake mix 2
sugar-free chocolate 2
date honey 2
kokum 2
gluten-free margarine 2
young ginger root 2
light hawaiian punch 2
squirrel 2
lamb leg chops 2
light chicken broth 2
boneless beef ribs 2
satsuma oranges 2
red raisins 2
bresaola 2
70% lean ground beef 2
sugar-free banana cream pudding mix 2
whole wheat honey graham crackers 2
green pickled jalapeno pepper 2
light fudge topping 2
torani orange syrup 2
cheese pizza 2
turkey burgers 2
cinnamon spread 2
angus steaks 2
italian olive oil 2
northern pike fillets 2
shrimp bouillon cubes 2

low-fat banana cream yogurt 2
cooked short-grain brown rice 2
fat-free strawberry frozen yogurt 2
lemon flavored ice tea mix 2
fat-free sugar-free vanilla pudding 2
ragu chunky tomato onion & garlic sauce 2
sugar-free ginger ale 2
scrod fish 2
dole classic coleslaw 2
fat-free cream cheese with chives 2
calamari steaks 2
raw red cabbage 2
mango ice cream 2
kraft zesty italian dressing 2
soft cracker bread 2
blachan 2
cooked converted rice 2
sandwich wrap 2
matzoh crackers 2
dark mustard 2
quorn tenders 2
sun-dried cherries 2
lemon concentrate 2
fish head 2
dooley liqueur 2
sour kimchee 2
kippered herring fillets 2
bone-in pork roast 2
sourdough hamburger buns 2
vanilla gelato 2
piquin chilies 2
whole grain crackers 2
rattlesnake 2
gluten-free vegetable stock 2
four-grain cereal 2
vegetarian chicken stock 2
langoustine 2
unsweetened chocolate chips 2
dry pasta shell macaroni 2
instant devil's food pudding 2
mackerel fillet 2
ham knuckle 2
fresh cumin seed 2
pink candy melts 2
mushroom an

mrs. grass homestyle vegetable recipe 1
fat-free tomato and herb salad dressing 1
low-fat powdered milk 1
glayva 1
barilla linguine 1
veal kidney 1
sliced kosher pickles 1
white-meat chicken quarters 1
cream of mushroom soup mix 1
cinnamon biscuits 1
candy pumpkins 1
onions in cream sauce 1
indian spicy mango pickle 1
frozen pot sticker 1
chocolate square 1
light honey dijon salad dressing 1
rootbeer float ice cream 1
country pork strips 1
nonfat berry yogurt 1
banana nut cheerios 1
blackened chicken seasoning 1
seedless blackberry spread 1
gluten-free chili powder 1
muskmelon 1
low-fat low-sodium chicken and rice soup 1
apricot jell-o 1
rusks 1
chili pepper pasta 1
yellow fin potatoes 1
peanut and ginger tofu 1
abalone steak 1
multi-grain flakes cereal with oat clusters cranberries and almonds 1
brown paper bags 1
capsicum salsa 1
sugar-free red jell-o 1
yogurt spread 1
beets with greens 1
mutton rib 1
dandelion root 1
epicure lemon dilly dip mix 1
graviera 1
johnsonville classic ital

nuttter butter bites 1
maple pepper seasoning with habanero 1
dole banana 1
thoor dhal 1
nestl toll house dark chocolate morsels 1
raspberry hot cocoa mix 1
pillsbury sugar cookie mix 1
honey mustard pretzels 1
raspberry-flavored wine coolers 1
pineapple orange drink mix 1
purple sweet potato 1
no-salt-added vegetables 1
ice cream sandwich 1
pommeau de normandie 1
jonagold apples 1
spearmint candy leaf 1
frozen broccoli with red peppers 1
artichoke dip 1
baby apricots 1
lemon-pepper linguine 1
low fat biscuits 1
cinnamon vodka 1
powdered eggs 1
bourbon biscuits 1
cheese-stuffed manicotti 1
dried papayas 1
prego ricotta and parmesan sauce 1
green pattypan squash 1
tomato tortilla 1
ritz honey butter crackers 1
clementines, zest of 1
oat-bran cereal 1
paper drink umbrellas 1
reduced-fat neufchatel cheese 1
shellfish stock 1
clean jars 1
roasted wattleseed 1
betty crocker chicken helper chicken and herb rice 1
boucheron cheese 1
georgia pecan halves 1
blackfish fillet 1
chicken and garlic

---

In [83]:
stopped_tokens = []
for item in one_list:
    tokens = word_tokenize(item)
    stopped_tokens.append(w.lower() for w in tokens if w not in stopwords_list)

In [76]:
stopped_tokens

[<generator object <genexpr> at 0x1a62979bf8>,
 <generator object <genexpr> at 0x1a7fc5d518>,
 <generator object <genexpr> at 0x1a7fc5d048>,
 <generator object <genexpr> at 0x1a7fc5d360>,
 <generator object <genexpr> at 0x1a7fc5d468>,
 <generator object <genexpr> at 0x1a22457150>,
 <generator object <genexpr> at 0x1a7fc5d150>,
 <generator object <genexpr> at 0x1a7fc5d570>,
 <generator object <genexpr> at 0x1a21f5f570>,
 <generator object <genexpr> at 0x1a7c8613b8>,
 <generator object <genexpr> at 0x1a7c861200>,
 <generator object <genexpr> at 0x1a545cd9e8>,
 <generator object <genexpr> at 0x10df01728>,
 <generator object <genexpr> at 0x10df010f8>,
 <generator object <genexpr> at 0x1a6ea6ae08>,
 <generator object <genexpr> at 0x1a6ea6ad58>,
 <generator object <genexpr> at 0x1a6ea6ac50>,
 <generator object <genexpr> at 0x1a6ea6aca8>,
 <generator object <genexpr> at 0x10de63f68>,
 <generator object <genexpr> at 0x1a3dfa0f10>,
 <generator object <genexpr> at 0x1a5126b3b8>,
 <generator obje

In [77]:
list(stopped_tokens[0])

['winter', 'squash']

In [84]:
lemma_tokens = [lemmatizer.lemmatize(w) for token in stopped_tokens for w in token]

In [85]:
lemma_tokens

['winter',
 'squash',
 'mexican',
 'seasoning',
 'mixed',
 'spice',
 'honey',
 'butter',
 'olive',
 'oil',
 'salt',
 'prepared',
 'pizza',
 'crust',
 'sausage',
 'patty',
 'egg',
 'milk',
 'salt',
 'pepper',
 'cheese',
 'beef',
 'yellow',
 'onion',
 'tomato',
 'tomato',
 'paste',
 'tomato',
 'soup',
 'rotel',
 'tomato',
 'kidney',
 'bean',
 'water',
 'chili',
 'powder',
 'cumin',
 'salt',
 'lettuce',
 'cheddar',
 'cheese',
 'spreadable',
 'cheese',
 'garlic',
 'herb',
 'new',
 'potato',
 'shallot',
 'parsley',
 'tarragon',
 'olive',
 'oil',
 'red',
 'wine',
 'vinegar',
 'salt',
 'pepper',
 'red',
 'bell',
 'pepper',
 'yellow',
 'bell',
 'pepper',
 'tomato',
 'juice',
 'apple',
 'cider',
 'vinegar',
 'sugar',
 'salt',
 'pepper',
 'clove',
 'oil',
 'cinnamon',
 'oil',
 'mustard',
 'milk',
 'vanilla',
 'ice',
 'cream',
 'apple',
 'juice',
 'concentrate',
 'apple',
 'fennel',
 'seed',
 'green',
 'olive',
 'olive',
 'garlic',
 'peppercorn',
 'orange',
 'rind',
 'orange',
 'juice',
 'red',
 

In [63]:
all_tokens = [_flatten(list(stopped_token)) for stopped_token in stopped_tokens]

In [64]:
all_tokens

[<generator object _flatten at 0x1a7fc5d308>,
 <generator object _flatten at 0x1a7fc5d620>,
 <generator object _flatten at 0x1a7fc5d5c8>,
 <generator object _flatten at 0x1a7fc5d2b0>,
 <generator object _flatten at 0x1a7fc5d258>,
 <generator object _flatten at 0x1a7fc5d200>,
 <generator object _flatten at 0x1a7fc5d1a8>,
 <generator object _flatten at 0x1a7fc5d0a0>,
 <generator object _flatten at 0x1a7fc5d4c0>,
 <generator object _flatten at 0x1a7fc5d3b8>,
 <generator object _flatten at 0x1a7fc5d6d0>,
 <generator object _flatten at 0x1a7fc5d728>,
 <generator object _flatten at 0x1a7fc5d780>,
 <generator object _flatten at 0x1a7fc5d7d8>,
 <generator object _flatten at 0x1a7fc5d830>,
 <generator object _flatten at 0x1a7fc5d888>,
 <generator object _flatten at 0x1a7fc5d8e0>,
 <generator object _flatten at 0x1a7fc5d938>,
 <generator object _flatten at 0x1a7fc5d990>,
 <generator object _flatten at 0x1a7fc5d9e8>,
 <generator object _flatten at 0x1a7fc5da40>,
 <generator object _flatten at 0x1

In [65]:
tokenized = []

for listed in one_list:
    tokenized.append(simple_preprocess(listed, min_len=2))

In [66]:
tokenized

[['winter', 'squash'],
 ['mexican', 'seasoning'],
 ['mixed', 'spice'],
 ['honey'],
 ['butter'],
 ['olive', 'oil'],
 ['salt'],
 ['prepared', 'pizza', 'crust'],
 ['sausage', 'patty'],
 ['eggs'],
 ['milk'],
 ['salt', 'and', 'pepper'],
 ['cheese'],
 ['ground', 'beef'],
 ['yellow', 'onions'],
 ['diced', 'tomatoes'],
 ['tomato', 'paste'],
 ['tomato', 'soup'],
 ['rotel', 'tomatoes'],
 ['kidney', 'beans'],
 ['water'],
 ['chili', 'powder'],
 ['ground', 'cumin'],
 ['salt'],
 ['lettuce'],
 ['cheddar', 'cheese'],
 ['spreadable', 'cheese', 'with', 'garlic', 'and', 'herbs'],
 ['new', 'potatoes'],
 ['shallots'],
 ['parsley'],
 ['tarragon'],
 ['olive', 'oil'],
 ['red', 'wine', 'vinegar'],
 ['salt'],
 ['pepper'],
 ['red', 'bell', 'pepper'],
 ['yellow', 'bell', 'pepper'],
 ['tomato', 'juice'],
 ['apple', 'cider', 'vinegar'],
 ['sugar'],
 ['salt'],
 ['pepper'],
 ['clove', 'oil'],
 ['cinnamon', 'oil'],
 ['dry', 'mustard'],
 ['milk'],
 ['vanilla', 'ice', 'cream'],
 ['frozen', 'apple', 'juice', 'concentrate

In [67]:
flat_tokenized = list(_flatten(tokenized))

In [68]:
ingred_tokens_freqdist = FreqDist(flat_tokenized)

In [69]:
ingred_tokens_freqdist.most_common(200)

[('salt', 126481),
 ('pepper', 119077),
 ('sugar', 94253),
 ('oil', 80876),
 ('cheese', 76540),
 ('fresh', 75568),
 ('garlic', 72372),
 ('butter', 71551),
 ('onion', 65240),
 ('ground', 65009),
 ('flour', 60384),
 ('cream', 52091),
 ('powder', 51763),
 ('water', 46562),
 ('sauce', 45942),
 ('juice', 45027),
 ('chicken', 44303),
 ('olive', 42941),
 ('milk', 42692),
 ('red', 40089),
 ('black', 38017),
 ('eggs', 35294),
 ('baking', 34681),
 ('lemon', 34358),
 ('green', 32589),
 ('vanilla', 31221),
 ('dried', 30930),
 ('cloves', 30647),
 ('white', 29626),
 ('egg', 29626),
 ('tomatoes', 27886),
 ('brown', 27184),
 ('and', 23250),
 ('vinegar', 22183),
 ('cinnamon', 21953),
 ('parsley', 20685),
 ('onions', 20250),
 ('beef', 19543),
 ('all', 19025),
 ('vegetable', 18761),
 ('purpose', 18746),
 ('wine', 17523),
 ('chocolate', 17423),
 ('of', 16892),
 ('mustard', 16400),
 ('extract', 15531),
 ('parmesan', 15492),
 ('rice', 15415),
 ('dry', 15403),
 ('broth', 15265),
 ('soda', 15256),
 ('celery',

In [70]:
ingred_tokens_freqdist.most_common()[:200:-1]

[('nepitella', 1),
 ('rosarita', 1),
 ('mixer', 1),
 ('stirrings', 1),
 ('namak', 1),
 ('yigandes', 1),
 ('polished', 1),
 ('arepa', 1),
 ('chefstyle', 1),
 ('sargento', 1),
 ('trumpet', 1),
 ('wham', 1),
 ('teifi', 1),
 ('collins', 1),
 ('dubonnet', 1),
 ('oak', 1),
 ('pineau', 1),
 ('kabob', 1),
 ('saba', 1),
 ('louis', 1),
 ('miao', 1),
 ('dou', 1),
 ('swizzle', 1),
 ('paddy', 1),
 ('clean', 1),
 ('espices', 1),
 ('sachet', 1),
 ('concerto', 1),
 ('petroleum', 1),
 ('kijafa', 1),
 ('garcia', 1),
 ('valentine', 1),
 ('elixir', 1),
 ('sobe', 1),
 ('konjac', 1),
 ('kvas', 1),
 ('caribe', 1),
 ('twisted', 1),
 ('turtles', 1),
 ('flatout', 1),
 ('victoria', 1),
 ('jonagold', 1),
 ('salute', 1),
 ('invert', 1),
 ('bolillos', 1),
 ('squeeze', 1),
 ('poha', 1),
 ('washing', 1),
 ('toffeepop', 1),
 ('tam', 1),
 ('tim', 1),
 ('yak', 1),
 ('rollo', 1),
 ('koolaid', 1),
 ('munchee', 1),
 ('skillets', 1),
 ('trappey', 1),
 ('raita', 1),
 ('pandanus', 1),
 ('loins', 1),
 ('umbrellas', 1),
 ('tarh

In [71]:
flat_tokenized

['winter',
 'squash',
 'mexican',
 'seasoning',
 'mixed',
 'spice',
 'honey',
 'butter',
 'olive',
 'oil',
 'salt',
 'prepared',
 'pizza',
 'crust',
 'sausage',
 'patty',
 'eggs',
 'milk',
 'salt',
 'and',
 'pepper',
 'cheese',
 'ground',
 'beef',
 'yellow',
 'onions',
 'diced',
 'tomatoes',
 'tomato',
 'paste',
 'tomato',
 'soup',
 'rotel',
 'tomatoes',
 'kidney',
 'beans',
 'water',
 'chili',
 'powder',
 'ground',
 'cumin',
 'salt',
 'lettuce',
 'cheddar',
 'cheese',
 'spreadable',
 'cheese',
 'with',
 'garlic',
 'and',
 'herbs',
 'new',
 'potatoes',
 'shallots',
 'parsley',
 'tarragon',
 'olive',
 'oil',
 'red',
 'wine',
 'vinegar',
 'salt',
 'pepper',
 'red',
 'bell',
 'pepper',
 'yellow',
 'bell',
 'pepper',
 'tomato',
 'juice',
 'apple',
 'cider',
 'vinegar',
 'sugar',
 'salt',
 'pepper',
 'clove',
 'oil',
 'cinnamon',
 'oil',
 'dry',
 'mustard',
 'milk',
 'vanilla',
 'ice',
 'cream',
 'frozen',
 'apple',
 'juice',
 'concentrate',
 'apple',
 'fennel',
 'seeds',
 'green',
 'olives

---

In [ ]:
big_list_names = []
for row in df_file['name']:
    big_list_names.append(ast.literal_eval(row))
one_list_names = list(_flatten(big_list))
one_list_names
len(one_list_names)
len(set(one_list_names))

In [ ]:
all_names = list(df_file['name'].values)

In [ ]:
replace = {'rotel': 'canned',
          'red bell pepper': 'bell_pepper',
          'yellow bell pepper': 'bell_pepper',
          'green bell pepper': 'bell_pepper',
          }

In [ ]:
brands = ['rotel', 'absolut', 'betty crocker', 'jello', 'diana', 'ener-g', 
          'del-monte', "hunt's", 'martha', 'goya', 'cracker barrel', 
          'hamburger helper', "mccormick's", 'pepperidge farm', 'knorr', 
          'godiva', 'hidden valley', 'tabasco', 'branston', "kellogg's", 
          'hodgson mill', 'kraft', 'johnsonville', 'jim beam', 'mccormick', 
          'equal', 'jell-o', 'jimmy dean', 'country bob', "smucker's", 
          'toblerone', 'gerber', 'nestle', 'nestl', 'malt-o-meal', 'triscuit', 
          'ragu', 'campbell', 'hormel', 'earth balance', 'pillsbury', "bird's eye", 
          "campbell's", "betty crocker's", 
         ]

In [ ]:
impute = ['vegan', 'vegetarian']